B1: load mode, encode doc

In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
import os
import re
import numpy as np

# Khởi tạo tokenizer và mô hình PhoBERT
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModel.from_pretrained("vinai/phobert-base")

# Hàm để chia tài liệu thành các đoạn nhỏ hơn
def chunk_document(text, max_length=100):
    tokens = tokenizer.tokenize(text)
    chunks = [tokens[i:i + max_length] for i in range(0, len(tokens), max_length)]
    return chunks

# Hàm để mã hóa đoạn văn bản
def encode_chunk(chunk):
    # Nếu chunk trống, trả về một vector không (zero vector) hoặc bỏ qua
    if len(chunk) == 0:
        return None
    inputs = tokenizer(" ".join(chunk), return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    chunk_vector = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return chunk_vector

# Hàm để mã hóa toàn bộ tài liệu
def encode_long_text(text):
    chunks = chunk_document(text)
    chunk_vectors = []
    for chunk in chunks:
        chunk_vector = encode_chunk(chunk)
        if chunk_vector is not None:  # Chỉ thêm các vector hợp lệ
            chunk_vectors.append(chunk_vector)
    return chunk_vectors

def load_documents_from_directory(directory):
    print('Loading documents from vnexpress directory')
    docs = []
    for file_name in os.listdir(directory):
        print('Đọc dữ liệu từ file: [{}]...'.format(file_name))
        file_path = os.path.join(directory, file_name)
        lines = []
        with open(file_path, 'r', encoding='utf-8') as fi:
            # reade file acording lines.
            for line in fi:
                # change to lower character and remove space at head/tail of line.
                line = line.lower().strip()
                # add to list
                lines.append(line)
        # create doc as text inline.
        doc = " ".join(lines)
        # clean, remove some spec word in head/tail.
        re.sub('\W+',' ', doc)
        docs.append(doc) 
    print('End documents from vnexpress directory')
    return docs

# Ví dụ danh sách các tài liệu dài
#documents = ["sạt lở đất ở Tà Xùa, Sơn La ngày 10/8. Ảnh: Ban chỉ huy quân sự huyện Bắc YênTrung tâm Dự báo Khí tượng Thủy văn quốc gia nhận định từ nay đến ngày 17/8, miền Bắc sẽ mưa lớn, cao điểm mưa 60-120 mm, có nơi trên 250 mm tập trung từ 11 đến 13/8. Lượng mưa lớn, tập trung vào chiều tối và đêm, trên phạm vi hẹp (ở không gian cấp tỉnh) nên khả năng cao gây lũ quét, sạt lở đất, ngập úng tại các khu vực trũng thấp, đặc biệt ở các tỉnh vùng núi phía Bắc.Trang Accuweather của Mỹ dự báo Hà Nội tuần tới phổ biến 26-32 độ C, riêng thứ năm nhiệt độ cao nhất trong ngày lên 34 độ. Điểm cao trên 1.500 m so với mực nước biển như Sa Pa (Lào Cai) nhiệt độ trong tuần 17-26 độ C.Miền Trung các tỉnh Thanh Hóa, Nghệ An chịu ảnh hưởng của rãnh áp thấp ở phía bắc nên đêm nay và chiều tối mai có mưa rào, giông. Vùng nắng nóng thu hẹp vào các tỉnh Hà Tĩnh - Bình Thuận với nhiệt độ cao nhất ngày phổ biến 35-37 độ, có nơi trên 37 độ C.Nam Bộ và Tây Nguyên từ nay đến 13/8 có mưa rào, giông về chiều tối. Các ngày 14-17/8, gió mùa tây nam suy yếu, mưa tiếp tục giảm.Nhiệt độ cao nhất ở Nam Bộ phổ biến 32-34 độ, Tây Nguyên 29-32 độ C. Từ ngày 14/7, nhiệt độ Nam Bộ tăng lên 33-35 độ, miền Đông Nam Bộ có nắng nóng cục bộ, Tây Nguyên 30-33 độ C.Miền Bắc vừa trải qua gần một tuần nắng nóng nhưng các tỉnh miền núi vẫn mưa rào, giông về chiều tối và đêm. Độ ẩm trong đất ở nhiều khu vực đã bão hòa nên nguy cơ sạt lở đất rất cao.Bộ Nông nghiệp và Phát triển nông thôn hôm qua tiếp tục yêu cầu UBND các tỉnh miền Bắc theo dõi diễn biến thời tiết, tổ chức kiểm tra khu vực có nguy cơ lũ quét, sạt lở. Các địa phương tổ chức canh gác ở ngầm tràn, khu vực đã hoặc có nguy cơ sạt lở, không cho người qua lại; sẵn sàng lực lượng, phương tiện cứu hộ, cứu nạn khi có yêu cầu.", 
#             "trước đó gần một ngày, CAS ra phán quyết chấp thuận vụ kiện của COSR về tranh cãi liên quan đến HC đồng thể dục dụng cụ biểu diễn sàn nữ. Số điểm ban đầu của Chiles cần được giữ nguyên là 13,666 điểm, vì thế Barbosu với 13,700 điểm nhận được HC đồng.Chung kết biểu diễn sàn đã diễn ra hôm 5/8, khi Chiles thi cuối và được trọng tài chấm 13,666 điểm. Trong khi Barbosu reo mừng, đội thể dục dụng cụ Mỹ khiếu nại lên các trọng tài, vì cho rằng họ đã chấm nhầm điểm độ khó của nữ VĐV 23 tuổi.Trong thể dục dụng cụ, mỗi phần thi của VĐV thường có điểm độ khó được xác định dựa trên các loại động tác, và điểm kỹ thuật dựa theo đánh giá của trọng tài. Điểm độ khó của Chiles đã được quy ước là 5,9 nhưng trọng tài mắc sai lầm nên chỉ chấm 5,8.Trọng tài sau đó chấp nhận khiếu nại của Mỹ, chấm lại điểm độ khó của Chiles từ",
#             ]
documents = load_documents_from_directory('/content/data-nlp/')


# Mã hóa và lưu trữ các vector biểu diễn
document_vectors = []
for doc in documents:
    vectors = encode_long_text(doc)
    document_vectors.append([v.tolist() for v in vectors])



ModuleNotFoundError: No module named 'transformers'

Bước 2: Mã Hóa Câu Truy Vấn

In [ ]:
# Câu truy vấn
query = "Trước thảm họa khiến khoảng 150 người thiệt mạng trong đám đông"

# Mã hóa câu truy vấn
query_tokens = tokenizer.tokenize(query)
query_inputs = tokenizer(" ".join(query_tokens), return_tensors="pt", padding=True, truncation=True, max_length=512)
with torch.no_grad():
    query_outputs = model(**query_inputs)
query_vector = query_outputs.last_hidden_state.mean(dim=1).squeeze().numpy()


Bước 3: Tính Toán Độ Tương Đồng

In [ ]:
# Hàm tính độ tương đồng cosine
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# Tính toán độ tương đồng giữa câu truy vấn và các đoạn của mỗi tài liệu
similarities = []
for doc_vectors in document_vectors:
    doc_similarities = [cosine_similarity(query_vector, vec) for vec in doc_vectors]
    max_similarity = max(doc_similarities)
    similarities.append(max_similarity)

# Sắp xếp các tài liệu theo độ tương đồng giảm dần
sorted_docs = sorted(zip(documents, similarities), key=lambda x: x[1], reverse=True)

# In ra các tài liệu có độ tương đồng cao nhất
result_doc = []
for doc, similarity in sorted_docs[:30]:  # Top 10 tài liệu tương đồng nhất
    result_doc.append(f"Tài liệu: {doc[:100]}... (độ tương đồng: {similarity:.4f})")
    print(f"Tài liệu: {doc[:100]}... (độ tương đồng: {similarity:.4f})")
    
